# Reflection and Blogpost Writing

## Setup

In [1]:
import os
api_key = os.environ["GEMINI_API_KEY"]
llm_config = { "model": "gemini-1.5-pro-latest", "api_key": api_key, "api_type": "google"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task ,"role": "user"}])

In [5]:
print(reply)

{'content': "**Hope AI: Artificial Intelligence with a Heart**\n\nHope AI isn't just another AI company; it represents a new wave of artificial intelligence focused on positive global impact.  By developing AI solutions for pressing issues like climate change, poverty, and disease, Hope AI aims to harness technology's power for good.  Their innovative projects, from predictive models for disaster relief to personalized education platforms, exemplify their commitment.  Hope AI believes in a future where technology empowers us all, creating a world brimming with hope and opportunity.  Learn more about their inspiring initiatives and how you can get involved at hopeai.com (fictional website).\n", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Hope AI: Illuminating the Path to Early Cancer Detection**

Hope AI is pioneering a new frontier in cancer diagnostics. Using advanced image analysis and machine learning, the technology analyzes readily available medical images, like CT scans, to detect cancerous growths at their earliest stages, even before they become visible to the human eye.  This innovative approach promises to revolutionize cancer screening, offering a non-invasive and potentially life-saving tool for early detection and intervention.  With Hope AI, the future of cancer care looks brighter, offering a beacon of hope for patients and medical professionals alike.


--------------------------------------------------------------------------------
Critic (to Writer):


## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Hope AI: Illuminating the Path to Early Cancer Detection**

Hope AI is pioneering a new frontier in cancer diagnostics. Using advanced image analysis and machine learning, the technology analyzes readily available medical images, like CT scans, to detect cancerous growths at their earliest stages, even before they become visible to the human eye.  This innovative approach promises to revolutionize cancer screening, offering a non-invasive and potentially life-saving tool for early detection and intervention.  With Hope AI, the future of cancer care looks brighter, offering a beacon of hope for patients and medical professionals alike.


--------------------------------------------------------------------------------

*******************

## Get the summary

In [15]:
print(res.summary)

**Hope AI: Advancing Early Cancer Detection Through AI-Powered Image Analysis**

Hope AI is a novel technology employing artificial intelligence and image analysis to assist medical professionals in detecting potential signs of early-stage cancer. By analyzing medical images like CT scans, Hope AI identifies subtle patterns that could indicate cancerous growths, potentially aiding in earlier diagnosis and intervention.  This AI-powered approach offers a new tool for cancer screening, working in concert with healthcare providers to enhance the diagnostic process. While access to this technology may vary, we are committed to equitable distribution and striving to make it accessible to all patients.  Learn more about Hope AI and its potential applications in early cancer detection [link].

